# Data Filtering and Aggregating

### PROJECT

### Цель:
Ответить на вопросы, применив различные способы группировки, фильтрации и агрегации данных.

In [28]:
# Импортируем необходимые библиотеки и установим количество отображаемых строк и столбцов по умолчанию
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)  # показывать больше колонок
pd.set_option('display.max_colwidth', 60)  # показывать полный текст в ячейке

from collections import Counter
import collections
import itertools

In [29]:
# Загрузим данные и посмотрим на датасет 
data = pd.read_csv('movie_bd_v5.csv')
display(data.sample(3))
display(data.info())
print(f"Размер датасета: {data.shape}\n")
print(f'\nКоличество уникальных значений: \n{data.nunique()}')
print('\nКоличество пропущенных значений в датасете: {}'.format(data.isna().sum().sum()))
display(data.describe())
data.describe(include='object') 

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
824,tt1068242,24000000,63543328,Footloose,Kenny Wormald|Julianne Hough|Dennis Quaid|Andie MacDowel...,Craig Brewer,There comes a time to cut loose.,City kid Ren MacCormack moves to a small town where rock...,113,Drama|Music|Romance,Spyglass Entertainment|MTV Films,10/6/2011,6.5,2011
730,tt1229238,145000000,694713380,Mission: Impossible - Ghost Protocol,Tom Cruise|Jeremy Renner|Simon Pegg|Paula Patton|Anil Ka...,Brad Bird,No Plan. No Backup. No Choice.,"In the 4th installment of the Mission Impossible series,...",133,Action|Thriller|Adventure,Paramount Pictures|Skydance Productions|Bad Robot|TC Pro...,12/7/2011,6.7,2011
1472,tt0472043,40000000,120175290,Apocalypto,Rudy Youngblood|Dalia HernÃ¡ndez|Jonathan Brewer|Morris ...,Mel Gibson,"When the end comes, not everyone is ready to go","Set in the Mayan civilization, when a man's idyllic pres...",139,Thriller|Action|Adventure|Drama,Icon Productions|Icon Entertainment International|Touchs...,12/7/2006,7.1,2006


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


None

Размер датасета: (1889, 14)


Количество уникальных значений: 
imdb_id                 1889
budget                   203
revenue                 1885
original_title          1887
cast                    1878
director                 957
tagline                 1880
overview                1889
runtime                  105
genres                   652
production_companies    1624
release_date            1435
vote_average              47
release_year              16
dtype: int64

Количество пропущенных значений в датасете: 0


,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


,imdb_id,original_title,cast,director,tagline,overview,genres,production_companies,release_date
count,1889,1889,1889,1889,1889,1889,1889,1889,1889
unique,1889,1887,1878,957,1880,1889,652,1624,1435
top,tt0369610,The Gift,Jennifer Lawrence|Josh Hutcherson|Liam Hemsworth|Woody H...,Steven Soderbergh,What could go wrong?,"Twenty-two years after the events of Jurassic Park, Isla...",Comedy,New Line Cinema,12/25/2014
freq,1,2,3,13,2,1,141,21,7


## Предобработка

In [6]:
# создадим словарь,  в котором будем хранить ответы на вопросы
answers = {} 

# Признак, обозначающий прибыль (разность между сборами и бюджетом фильма) 
data['profit'] = data['revenue'] - data['budget'] 

# Признак, содержащий количество буквенных знаков в названии фильма
#data.original_title.apply(lambda x: len(x))
data['title_length'] = data['original_title'].str.len()

# Признак, содержащий количество буквенных знаков в описании фильма
data['overview_length'] = data['overview'].str.len()


In [7]:
# Создадим функцию для получения словаря с количеством фильмов по жанрам из исходной таблицы

def counter_dict(data, dict, field):
    for subdata in data[field]:
        for thing in subdata.split('|'):
            if thing in dict:
                dict[thing] += 1
            else: dict[thing] = 1
    return dict 


# 1. У какого фильма из списка самый большой бюджет?

In [8]:
# Предоставляем решение 
data.loc[data.budget == data.budget.max()][['imdb_id', 'original_title']]

,imdb_id,original_title
723,tt1298650,Pirates of the Caribbean: On Stranger Tides


In [9]:
# Можно предоставить также другой способ решения этого вопроса
data[data['budget'].isin([data['budget'].max()])][['imdb_id', 'original_title']]

,imdb_id,original_title
723,tt1298650,Pirates of the Caribbean: On Stranger Tides


In [10]:
# Еще один способ решения
data.query('budget == budget.max()')[['imdb_id', 'original_title']]

,imdb_id,original_title
723,tt1298650,Pirates of the Caribbean: On Stranger Tides


In [11]:
# Еще один способ решения
data[data['budget'] == data['budget'].max()][['imdb_id', 'original_title']]

,imdb_id,original_title
723,tt1298650,Pirates of the Caribbean: On Stranger Tides


In [12]:
# Помещаем в словарь номер вопроса и соответствующий ответ 

answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'  # "+"


# 2. Какой из фильмов самый длительный (в минутах)?

In [13]:
data.loc[data['runtime'] == data['runtime'].max()][['imdb_id','original_title','runtime']]

,imdb_id,original_title,runtime
1157,tt0279111,Gods and Generals,214


In [14]:

answers['2'] = 'Gods and Generals (tt0279111)'    # "+"


# 3. Какой из фильмов самый короткий (в минутах)?





In [15]:
data[data['runtime'] == data['runtime'].min()][['imdb_id','original_title','runtime']]

,imdb_id,original_title,runtime
768,tt1449283,Winnie the Pooh,63


In [16]:

answers['3'] = 'Winnie the Pooh (tt1449283)'    # "+"


# 4. Какова средняя длительность фильмов?


In [17]:
round(data['runtime'].mean())

110

In [18]:

answers['4'] = '110'


# 5. Каково медианное значение длительности фильмов? 

In [19]:
round(data['runtime'].median())

107

In [20]:

answers['5'] = '107'    # "+"


# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [21]:
data.loc[data['profit'] == data['profit'].max()][[ 'imdb_id','original_title']]

,imdb_id,original_title
239,tt0499549,Avatar


In [22]:
# Другой способ решения
data[data['profit'].isin([data['profit'].max()])][[ 'imdb_id','original_title']]

,imdb_id,original_title
239,tt0499549,Avatar


In [23]:

answers['6'] = 'Avatar (tt0499549)'   # "+"


# 7. Какой фильм самый убыточный? 

In [24]:
data[data['profit'] == data['profit'].min()][['imdb_id','original_title']]

,imdb_id,original_title
1245,tt1210819,The Lone Ranger


In [25]:
# Другой способ решения
data[data['profit'].isin([data['profit'].min()])][['imdb_id','original_title']]

,imdb_id,original_title
1245,tt1210819,The Lone Ranger


In [26]:

answers['7'] = 'The Lone Ranger (tt1210819)'   # "+"


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [27]:
# data.query('revenue > budget').original_title.count()
# data.loc[data.revenue > data.budget].imdb_id.nunique()

data[data['revenue'] > data['budget']]['imdb_id'].count()

1478

In [28]:
# Другой способ решения
len(data[data['profit'] >0])

1478

In [29]:

answers['8'] = '1478'   # "+"


# 9. Какой фильм оказался самым кассовым в 2008 году?

In [30]:
data[data.release_year == 2008].sort_values(['revenue'], ascending=False)[['imdb_id','original_title']].iloc[[0]]

,imdb_id,original_title
599,tt0468569,The Dark Knight


In [31]:
# Другой способ решения
data[data['revenue'].isin([data[data['release_year'] == 2008]['revenue'].max()])][['imdb_id', 'original_title']]

,imdb_id,original_title
599,tt0468569,The Dark Knight


In [32]:

answers['9'] = 'The Dark Knight(tt0468569)'   # "+"


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [38]:
data[data['release_year'].between(2012,2014)].sort_values('profit', ascending = True)[['imdb_id', 'original_title']].iloc[[0]]

,imdb_id,original_title
1245,tt1210819,The Lone Ranger


#### ДРУГИЕ ВАРИАНТЫ РЕШЕНИЯ:  
фильтрация с помощью query, between, isin, функции groupby и простая фильтрация

In [36]:
data.query('2012 <= release_year <= 2014').sort_values('profit')[['imdb_id', 'original_title']].iloc[[0]]

,imdb_id,original_title
1245,tt1210819,The Lone Ranger


In [36]:
data[(data['release_year'].between(2012,2014)) & (data['profit'] == data['profit'].min())][['imdb_id', 'original_title']]

,imdb_id,original_title
1245,tt1210819,The Lone Ranger


In [39]:
# data[data['profit'].isin([data.query('2012 <= release_year <= 2014')['profit'].min()])][['imdb_id', 'original_title']]

data[data.profit.isin([data[data.release_year.between(2012,2014)].profit.min()])][['imdb_id','original_title']]

,imdb_id,original_title
1245,tt1210819,The Lone Ranger


In [41]:
data[data.profit.isin([data.groupby(data.release_year.between(2013,2014)).profit.min()[1]])][['imdb_id', 'original_title']]

,imdb_id,original_title
1245,tt1210819,The Lone Ranger


In [42]:

answers['10'] = 'The Lone Ranger (tt1210819)'    # "+"


# 11. Какого жанра фильмов больше всего?

In [43]:
data_2 = data.copy()
data_2['genres'] = data_2['genres'].str.split('|')
genres = data_2.explode('genres')['genres'] 
genres_counter = Counter(genres)
genres_counter.most_common(1)[0][0]

'Drama'

ВАРИАНТ 2

In [44]:
genres.value_counts().index[0]

'Drama'

In [45]:

answers['11'] = 'Drama'    # "+"


# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

ВАРИАНТ 1  
**Решение с помощью функции counter_dict (помещена в предобработку)**

In [46]:
genres_count_dict = {}
data_profit = data[data['profit'] > 0]
genres_count_dict = counter_dict(data_profit, genres_count_dict, 'genres')
sorted(genres_count_dict.items(), reverse=True, key=lambda x: x[1])[0]

('Drama', 560)

ВАРИАНТ 2  
**Решение с помощью модуля collections**

In [47]:
counter = collections.Counter()
genres = data[data['profit'] > 0]['genres']
for genre in genres.str.split('|').sum():
    counter[genre] += 1
counter.most_common(1)[0][0]

'Drama'

#### Решение с помощью  str.split, explode и value_counts

In [48]:
data_12 = data.copy()
data_12.genres = data_12.genres.str.split('|')
data_12 = data_12.explode('genres')

# data_12[data_12.profit >0].groupby('genres').original_title.count().sort_values(ascending=False)
data_12.loc[data_12.profit>0].genres.value_counts().sort_values(ascending=False)[:1]  

Drama    560
Name: genres, dtype: int64

In [49]:

answers['12'] = 'Drama'    # "+"


# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [57]:
data_13 = data.copy()
data_13['director'] = data_13.director.str.split('|')
data_13.explode('director').groupby('director', as_index=False)['revenue'].sum()  \
                .sort_values('revenue', ascending=False)  \
                .rename(columns = {'revenue' : 'sum_revenue'})[:1]

,director,sum_revenue
727,Peter Jackson,6490593685


In [58]:

answers['13'] = 'Peter Jackson'    # "+"


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [59]:
data_14 = data.copy()
data_14['genres'] = data_14['genres'].apply(lambda value: value.split('|'))
data_14['director'] = data_14['director'].apply(lambda value: value.split("|"))
data_14 = data_14.explode('genres')
data_14 = data_14.explode('director')
data_14[data_14['genres'] == 'Action']['director'].value_counts(ascending = False).index[0]

'Robert Rodriguez'

ВАРИАНТ 2  
**Решение с помощью str.split, explode и str.match**

In [60]:
data_14 = data.copy()
data_14.director = data_14.director.str.split('|')
data_14 = data_14.explode('director')
data_14[data_14.genres.str.contains('Action')].groupby('director').original_title.count().sort_values(ascending=False).index[0]

'Robert Rodriguez'

In [61]:

answers['14'] = 'Robert Rodriguez'    # "+"


# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

ВАРИАНТ 1  
**Решение с помощью функций explode и groupby**

In [62]:
data_15 = data.copy()
data_15['cast'] = data_15[data_15['release_year'] == 2012]['cast'].str.split('|')
data_15.explode('cast').groupby(['cast'])['revenue'].sum().sort_values(ascending=False).index[0]

'Chris Hemsworth'

ВАРИАНТ 2  
**Решение с помощью функций concat, explode и groupby**

In [62]:
data_15 = data.copy()
movies_in_2012 =  data_15[data_15['release_year'] == 2012]
cast_revenue_2012 = pd.concat([movies_in_2012['cast'].str.split('|'), movies_in_2012['revenue']], axis = 1).explode('cast')
cast_revenue_2012.groupby('cast')['revenue'].sum().sort_values(ascending = False).index[0]

'Chris Hemsworth'

In [63]:

answers['15'] = 'Chris Hemsworth'   # '+'


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [64]:
data_16 = data.copy()
data_16['cast'] = data_16[data_16['budget'] >= data_16['budget'].mean()].cast.str.split('|')

In [65]:
# data_16.explode('cast').cast.value_counts(ascending  = False)[:1]

data_16.explode('cast').groupby(['cast'])['imdb_id'].count().sort_values(ascending=False).index[0]

'Matt Damon'

In [66]:

answers['16'] = 'Matt Damon'   # '+'


# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [67]:
data_17 = data.copy()
genres_NC = data_17[data_17['cast'].str.contains('Nicolas Cage', na=False)]['genres'].str.split('|').explode()
Counter(genres_NC).most_common(1)[0][0]

'Action'

In [68]:

answers['17'] = 'Action'   # '+'


# 18. Самый убыточный фильм от Paramount Pictures

ВАРИАНТ 1 

In [70]:
# data[data.production_companies.str.contains('Paramount Pictures')].groupby('original_title').profit.min().sort_values()[:1]

data[data['production_companies'].str.contains('Paramount Pictures', na=False)].sort_values('profit')['original_title'].iloc[0]

'K-19: The Widowmaker'

ВАРИАНТ 2

In [73]:
prm_films = data[data['production_companies'].str.contains('Paramount Pictures')]
prm_films.loc[prm_films['profit'] == prm_films['profit'].min()]['original_title'].values[0]

'K-19: The Widowmaker'

In [72]:

answers['18'] = 'K-19: The Widowmaker (tt0267626)'   # '+'


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [74]:
data.groupby('release_year')['revenue'].sum().sort_values(ascending = False).index[0]

2015

In [75]:

answers['19'] = '2015'   # '+'


# 20. Какой самый прибыльный год для студии Warner Bros?

In [76]:
data[data['production_companies'].str.contains('Warner Bros.', na=False)].groupby(['release_year'])['profit'].sum()  \
        .sort_values(ascending = False).index[0]

2014

In [77]:

answers['20'] = '2014'   # '+'


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [80]:
# data_21['release_month'] = data_21.release_date.str.split('/').apply(lambda x:x[0])
# data_21['release_month'] = data_21['release_date'].apply(lambda x: x.split('/')[0])
# data_21['release_month'] = data_21['release_date'].apply(lambda x: x[:x.find('/')])

data_21 = data.copy()
data_21['release_date_month'] = data_21['release_date'].str.split('/').str.get(0)
data_21.groupby('release_date_month')['release_year'].count().sort_values(ascending=False).index[0]

'9'

ВАРИАНТ 2  
**Решение с помощью трансформации  to_datetime**

In [81]:
data_21 = data.copy()
data_21['month'] = pd.to_datetime(data_21.release_date).dt.month
data_21.groupby(['month']).original_title.count().sort_values(ascending=False)[:1]

month
9    227
Name: original_title, dtype: int64

In [82]:

answers['21'] = 'сентябрь'   # '+'


# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

ВАРИАНТ 1  
**Решение через lambda  between**

In [90]:
data_22 = data.copy()
data_22['release_date_month'] = data_22.release_date.apply(lambda x: int(x[:x.find('/')]))
data_22[data_22['release_date_month'].between(6, 8)].original_title.count()

450

ВАРИАНТ 2  
**Решение через трансформацию to_datetime и запрос query**

In [79]:
data_22 = data.copy()
data_22['month'] = pd.to_datetime(data_22.release_date).dt.month
data_22.query('month.between(6,8)').original_title.count()

450

ВАРИАНТ 3

In [87]:
data_22 = data.copy()
data_22['release_date_month'] = data_22['release_date'].str.split('/').str.get(0)
data_22[(data_22['release_date_month']=='6')|(data_22['release_date_month']=='7')|(data_22['release_date_month']=='8')]['imdb_id'].count()

450

In [104]:

answers['22'] = '450'   # '+'


# 23. Для какого режиссера зима – самое продуктивное время года? 

In [86]:
# data_23['month'] = data_23.release_date.apply(lambda x: x.split('/')[0])
# data_23['month'] = data_23['release_date'].str.split('/').str.get(0)

data_23 = data.copy()
data_23['month'] = data_23.release_date.apply(lambda x: x[:x.find('/')])
data_23.query("month in ['12','1','2']").groupby('director').original_title.count().sort_values(ascending=False)[:1]

director
Peter Jackson    7
Name: original_title, dtype: int64

In [87]:

answers['23'] = "Peter Jackson"   # '+'


# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [93]:
# Создание столбца  title_length  помещено в предобработку в начале проекта
data_24 = data.copy()
data_24['production_companies'] = data_24['production_companies'].str.split('|')
data_24.explode('production_companies').groupby('production_companies', as_index=False)  \
        .title_length.mean().sort_values('title_length', ascending=False)[:3]

,production_companies,title_length
583,Four By Two Productions,83.0
817,"Jim Henson Company, The",59.0
446,Dos Corazones,47.0


In [94]:

answers['24'] = "Four By Two Productions"   # '+'


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [95]:
data25 = data.copy()
data25['production_companies'] = data25['production_companies'].str.split('|')
data25.explode('production_companies')  \
            .groupby(['production_companies'])['overview_length'].mean().sort_values(ascending=False)[:3]

production_companies
Midnight Picture Show               1000.0
Room 9 Entertainment                 964.0
Brookwell-McNamara Entertainment     936.0
Name: overview_length, dtype: float64

In [96]:

answers['25'] = "Midnight Picture Show"   # '+'


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

ВАРИАНТ 1

In [97]:
data26 = data.copy()
data26.vote_average.sort_values(ascending=False)[:round(0.1*len(data26))].index.tolist()
data26.loc[data26.vote_average.sort_values(ascending=False)[:round(0.1*len(data26))].index.tolist(),['original_title','vote_average']]

,original_title,vote_average
599,The Dark Knight,8.1
118,Interstellar,8.0
125,The Imitation Game,8.0
9,Inside Out,8.0
34,Room,8.0
...,...,...
166,Chef,7.2
1591,Crash,7.1
705,The Express,7.1
1358,Match Point,7.1


ВАРИАНТ 2

In [98]:
data26.loc[data26.vote_average > data26.vote_average.quantile(0.99)].sort_values('vote_average', ascending=False)[['original_title', 'vote_average']]

,original_title,vote_average
599,The Dark Knight,8.1
9,Inside Out,8.0
34,Room,8.0
118,Interstellar,8.0
125,The Imitation Game,8.0
119,Guardians of the Galaxy,7.9
128,Gone Girl,7.9
138,The Grand Budapest Hotel,7.9
370,Inception,7.9
872,The Pianist,7.9


In [99]:

answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'   # '+'


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [100]:
data27 = data.copy()
data27['cast'] = data27.cast.str.split('|')
data27['cast'] = data27.cast.apply(lambda x: list(itertools.combinations(sorted(x), 2)))
data27.explode('cast').cast.value_counts()

(Daniel Radcliffe, Emma Watson)        8
(Daniel Radcliffe, Rupert Grint)       8
(Emma Watson, Rupert Grint)            8
(Helena Bonham Carter, Johnny Depp)    6
(Ben Stiller, Owen Wilson)             6
                                      ..
(Bryan Brown, Essie Davis)             1
(Bryan Brown, Hugh Jackman)            1
(Bryan Brown, Nicole Kidman)           1
(David Wenham, Essie Davis)            1
(Susan Ward, Tess Harper)              1
Name: cast, Length: 17943, dtype: int64

In [101]:

answers['27'] = 'Daniel Radcliffe, Rupert Grint'   # '+'


# Submission

In [105]:
# Посмотрим ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': '110',
 '5': '107',
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': '1478',
 '9': 'The Dark Knight(tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': 'сентябрь',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe, Rupert Grint',
 '22': '450'}

In [106]:
# Дополнительная проверка
len(answers) == 27

True